# 日本語テキストのベクトル化

- 日本語テキストに対する前処理を preprocess() として定義
- CountVectorizer, TfidfVectorizer から preprocess() を呼び出し

In [2]:
import pandas as pd
import re
import spacy

# 日本語モデル
nlp = spacy.load('ja_core_news_lg')

# フィードデータの読み込み、確認
feeds = pd.read_csv('data/output_jp.csv')

# title と summary を結合
# str.cat() により複数列の文字列を結合
# - sep=' ': 間に挟む文字列
# - na_rep='': NaN は空文字列に変換（指定しないと結合結果が NaN になる）
feeds['text'] = feeds['title'].str.cat(feeds['summary'], sep='。', na_rep='')

# 不要になった列を削除した処理用の DataFrame
df = feeds.drop(['title', 'summary'], axis=1)

# 確認
df.head()

,url,text
0,https://www.nhk.or.jp/rss/news/cat0.xml,大阪 クリニック放火事件からまもなく1年 遺族が手記公開。大阪のビルでクリニックが放火され、...
1,https://www.nhk.or.jp/rss/news/cat0.xml,ワールドカップ 日本 森保監督 今大会の成果と今後の課題は。サッカーのワールドカップカタール...
2,https://www.nhk.or.jp/rss/news/cat0.xml,ロシア空軍基地に“ウクライナ軍の無人機攻撃” 大きな打撃か。ロシア国内の複数の空軍基地で爆発...
3,https://www.nhk.or.jp/rss/news/cat0.xml,サッカー日本代表 たどり着いたもう1つの“新しい景色”。11月20日に開幕したサッカーのワー...
4,https://www.nhk.or.jp/rss/news/cat0.xml,北朝鮮 日本海向け約100発砲撃 2日連続 “米韓両軍への警告”。韓国軍は、北朝鮮が6日、日...


### 日本語テキストに対する前処理

preprocess(text) として定義
- 表記の正規化
- トークン化（形態素解析）
- ストップワードの除去
- 見出し語化

In [3]:
# 不要な単語を除去
# - ストップワード (is_stop)
# - いくつかの品詞
#     AUX: 助動詞
#     PUNCT: 句読点
#     SPACE: 空白文字
#     SYM: 記号
#     X: その他
# - うまく取り除けない単語や文字
stop_pos = ['AUX', 'PUNCT', 'SPACE', 'SYM', 'X']
stop_words = ['.']

def token_to_add(w):
    t = w.text    # 単語
    p = w.pos_    # 品詞
    l = w.lemma_  # 原型

    # ストップワードは None を返す
    if w.is_stop:
        return None
    if p in stop_pos:
        return None
    if l in stop_words:
        return None

    if len(l) == 0:
        return t
    return l

def preprocess(text):
    tokens = []
    
    for w in nlp(text):
        t = token_to_add(w)
        if t is not None:
            tokens.append(t)

    # トークンのリストを返す
    return tokens

### テキストのベクトル化 (1)

- Bag of Words (BoW)

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

# 例として最初の2行のテキストを処理
text_list = [df['text'].iloc[0], df['text'].iloc[1]]

# 初期化
# - tokenizer=preprocess: トークン化処理に上で定義した preprocess を使用することを指定
vectorizer = CountVectorizer(tokenizer=preprocess)

# ベクトル化
vector = vectorizer.fit_transform(text_list)

# 1行目
print(text_list[0])
print(vector[0])

大阪 クリニック放火事件からまもなく1年 遺族が手記公開。大阪のビルでクリニックが放火され、巻き込まれた26人が亡くなった事件からまもなく1年となるのを前に、被害者の遺族2人が報道陣に手記を寄せました。「いまだに現実を受け入れられていません」などと心情を記し、犯罪被害者の遺族への国の支援が十分でない実情を訴えています。
  (0, 29)	2
  (0, 6)	2
  (0, 41)	2
  (0, 12)	2
  (0, 0)	2
  (0, 33)	2
  (0, 57)	3
  (0, 38)	2
  (0, 20)	1
  (0, 10)	1
  (0, 32)	1
  (0, 55)	1
  (0, 1)	1
  (0, 14)	1
  (0, 13)	1
  (0, 21)	1
  (0, 51)	2
  (0, 50)	2
  (0, 2)	1
  (0, 27)	1
  (0, 58)	1
  (0, 31)	1
  (0, 3)	1
  (0, 48)	1
  (0, 24)	1
  (0, 19)	1
  (0, 34)	1
  (0, 52)	1
  (0, 47)	1
  (0, 26)	1
  (0, 40)	1
  (0, 22)	1
  (0, 30)	1
  (0, 53)	1


In [5]:
# ベクトルの単語との対応
for i in vector[0].indices:
    # print()
    # - end=' ': 改行の代わりに空白を出力
    print(vectorizer.get_feature_names_out()[i], end=' ')

大阪 クリニック 放火 事件 1 年 遺族 手記 公開 ビル 巻く 込む 26 人 亡くなる 前 被害 者 2人 報道 陣 寄せる いまだ 現実 受ける 入れる 心情 記す 犯罪 国 支援 十分 実情 訴える 

In [6]:
# 単語と頻度
[[vectorizer.get_feature_names_out()[i], vector[0, i]] for i in vector[0].indices]

[['大阪', 2],
 ['クリニック', 2],
 ['放火', 2],
 ['事件', 2],
 ['1', 2],
 ['年', 2],
 ['遺族', 3],
 ['手記', 2],
 ['公開', 1],
 ['ビル', 1],
 ['巻く', 1],
 ['込む', 1],
 ['26', 1],
 ['人', 1],
 ['亡くなる', 1],
 ['前', 1],
 ['被害', 2],
 ['者', 2],
 ['2人', 1],
 ['報道', 1],
 ['陣', 1],
 ['寄せる', 1],
 ['いまだ', 1],
 ['現実', 1],
 ['受ける', 1],
 ['入れる', 1],
 ['心情', 1],
 ['記す', 1],
 ['犯罪', 1],
 ['国', 1],
 ['支援', 1],
 ['十分', 1],
 ['実情', 1],
 ['訴える', 1]]

### テキストのベクトル化 (2)

- TF-IDF

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 例として最初の2行のテキストを処理
text_list = [df['text'].iloc[0], df['text'].iloc[1]]

# 初期化
# - tokenizer=preprocess: トークン化処理に上で定義した preprocess を使用することを指定
vectorizer = TfidfVectorizer(tokenizer=preprocess)

# ベクトル化
vector = vectorizer.fit_transform(text_list)

# 1行目
print(text_list[0])
print(vector[0])

大阪 クリニック放火事件からまもなく1年 遺族が手記公開。大阪のビルでクリニックが放火され、巻き込まれた26人が亡くなった事件からまもなく1年となるのを前に、被害者の遺族2人が報道陣に手記を寄せました。「いまだに現実を受け入れられていません」などと心情を記し、犯罪被害者の遺族への国の支援が十分でない実情を訴えています。
  (0, 53)	0.12214667990407793
  (0, 30)	0.12214667990407793
  (0, 22)	0.12214667990407793
  (0, 40)	0.12214667990407793
  (0, 26)	0.12214667990407793
  (0, 47)	0.12214667990407793
  (0, 52)	0.12214667990407793
  (0, 34)	0.12214667990407793
  (0, 19)	0.12214667990407793
  (0, 24)	0.12214667990407793
  (0, 48)	0.12214667990407793
  (0, 3)	0.12214667990407793
  (0, 31)	0.12214667990407793
  (0, 58)	0.12214667990407793
  (0, 27)	0.12214667990407793
  (0, 2)	0.12214667990407793
  (0, 50)	0.24429335980815586
  (0, 51)	0.24429335980815586
  (0, 21)	0.12214667990407793
  (0, 13)	0.12214667990407793
  (0, 14)	0.12214667990407793
  (0, 1)	0.12214667990407793
  (0, 55)	0.12214667990407793
  (0, 32)	0.12214667990407793
  (0, 10)	0.12214667990407793
  (0, 20)	0.12214667990407793
  (0, 38)	0.24429335980815586
  (0, 57)	0.36644003971223377
  

In [8]:
# ベクトルの単語との対応
for i in vector[0].indices:
    # print()
    # - end=' ': 改行の代わりに空白を出力
    print(vectorizer.get_feature_names_out()[i], end=' ')

訴える 実情 十分 支援 国 犯罪 記す 心情 入れる 受ける 現実 いまだ 寄せる 陣 報道 2人 者 被害 前 亡くなる 人 26 込む 巻く ビル 公開 手記 遺族 年 1 事件 放火 クリニック 大阪 

In [9]:
# 単語とTF-IDF
[[vectorizer.get_feature_names_out()[i], vector[0, i]] for i in vector[0].indices]

[['訴える', 0.12214667990407793],
 ['実情', 0.12214667990407793],
 ['十分', 0.12214667990407793],
 ['支援', 0.12214667990407793],
 ['国', 0.12214667990407793],
 ['犯罪', 0.12214667990407793],
 ['記す', 0.12214667990407793],
 ['心情', 0.12214667990407793],
 ['入れる', 0.12214667990407793],
 ['受ける', 0.12214667990407793],
 ['現実', 0.12214667990407793],
 ['いまだ', 0.12214667990407793],
 ['寄せる', 0.12214667990407793],
 ['陣', 0.12214667990407793],
 ['報道', 0.12214667990407793],
 ['2人', 0.12214667990407793],
 ['者', 0.24429335980815586],
 ['被害', 0.24429335980815586],
 ['前', 0.12214667990407793],
 ['亡くなる', 0.12214667990407793],
 ['人', 0.12214667990407793],
 ['26', 0.12214667990407793],
 ['込む', 0.12214667990407793],
 ['巻く', 0.12214667990407793],
 ['ビル', 0.12214667990407793],
 ['公開', 0.12214667990407793],
 ['手記', 0.24429335980815586],
 ['遺族', 0.36644003971223377],
 ['年', 0.24429335980815586],
 ['1', 0.1738167375332345],
 ['事件', 0.24429335980815586],
 ['放火', 0.24429335980815586],
 ['クリニック', 0.24429335980815586],
 ['大阪', 